In [1]:
import bs4
import requests
import pandas as pd
import numpy as np
import time
import os
from urllib.request import urlopen
from IPython.display import clear_output

- 네이버 영화 140자 평 데이터 수집.
- 소스보기 했을 때 원하는 데이터가 없다면 iframe 유무 확인.
- iframe의 src 속성의 주소를 요청해서 원하는 데이터가 있는지 확인.

In [2]:
# 요청 함수
def getSource(site) :
    # 헤더 정보
    header_info = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36'
    }
    
    # 요청한다.
    response = requests.get(site, headers = header_info)
    # print(response.text)
    
    # bs4 객체 생성
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    return soup

In [3]:
def getMovieCode() :
    site = 'https://movie.naver.com/movie/running/current.nhn'
    soup = getSource( site )
   # print(soup)
    
    # ul 태그를 가져온다. 
    a1 = soup.select_one('#content > div.article > div:nth-child(1) > div.lst_wrap > ul')
   # print(a1)
    
   # li 태그들을 가져옴.
    a2 = a1.select('li')
   # print(a2)
    
    # 코드들을 담을 리스트
    code_list = []
    
    
    # li 태그의 수 만큼 반복.
    for a3 in a2 :
        a4 = a3.select_one('dl > dt > a')
        # print(a4)
        
        # href 속성값을 가져옴.
        href = a4.attrs['href']
        # print(href)
        
        # =를 기준으로 문자열을 잘라내여 코드값만 가져옴.
        data1 = href.split('=')[1]
        # print(data1)
        
        code_list.append(data1.strip())
        
    return code_list

# k100 = getMovieCode()    
# print(k100)

In [4]:
# 한 페이지의 데이터를 수집해 저장하는 함수
def getData(soup, page) :
    
    data_dict = {
        '평점'      : [],
        '네티즌 평' : [],
        '작성자'    : [],
        '작성시간'  : [],
    }
    
    
    # ul 태그를 가져온다.
    a1 = soup.select_one('body > div > div > div.score_result > ul')
    # print(a1)
    
    # li 태그들을 가져온다.
    a2 = a1.select('li')
    # print(a2)
    
    # li 태그의 수 만큼 반복한다.
    for idx, a3 in enumerate(a2) :
        # 평점을 가져온다.
        a4 = a3.select_one('div.star_score > em')
        data1 = a4.text.strip()
        # print(data1)

        # 후기 글을 가져온다.
        a5 = a3.select_one(f'#_filtered_ment_{idx}')
        data2 = a5.text.strip()
        # print(data2)
        
        #_filtered_ment_0
        #_filtered_ment_5
        #_filtered_ment_3
        
        
        # 작성자를 가져온다.
        a6 = a3.select_one('div.score_reple > dl > dt > em:nth-child(1) > a > span')
        data3 = a6.text.strip()
        # print(data3)
        
        
        # 작성 날짜를 가져온다.
        a7 = a3.select_one('div.score_reple > dl > dt > em:nth-child(2)')
        data4 = a7.text.strip()
        # print(data4)
        
        
       # print(data1)
       # print(data2)
       # print(data3)
       # print(data4)
        data_dict['평점'].append(data1)
        data_dict['네티즌 평'].append(data2)
        data_dict['작성자'].append(data3)
        data_dict['작성시간'].append(data4)
        
        # 데이터 프레임 생성
        df1 = pd.DataFrame(data_dict)
        # display(df1)
        
        if os.path.exists('data3.csv') == False :
            # 파일이 없을 경우
            df1.to_csv('data3.csv', encoding='utf-8-sig', index=False)
            
        else :
            df1.to_csv('data3.csv', encoding='utf-8-sig', index=False, header=False, mode='a')
            
              
    # 전체 페이지의 수를 계산한다.
    a8 = soup.select_one('body > div > div > div.score_total > strong > em')
    data5 = a8.text.strip()
    # print(data5)   
    # 쉼표 제거.
    data5 = data5.replace(',','')  
    # 전체페이지 수를 계산.
    total_page = int(data5)//10
    if int(data5) % 10 > 0 :
        total_page +=1
    # print(total_page)
    
    
    # 테스트용 코드
    total_page = 5
    
    if total_page > page :
        # 전체 페이지 수가 현재 페이지 번호 보다 크면...
        return True
    else:
        return False
    

In [5]:
# 영화 코드들을 가져옴.
movie_code_list = getMovieCode()

# 테스트용 코드
movie_code_list = movie_code_list[:3]


for movie_code in movie_code_list :
    page = 1

    while True :
        time.sleep(1)
        clear_output(wait=True)
        print( f'{movie_code} : {page} 수집중')
        site = f'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code={movie_code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={page}'
        soup = getSource(site)
        chk  = getData(soup, page)

        if chk == True :
            page +=1
        else :
            break
print('수집완료')

184518 : 5 수집중
수집완료
